In [6]:
# Install the required packages
# !pip install pyrealsense2

import cv2
import numpy as np
import pyrealsense2 as rs

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        
        if not depth_frame or not color_frame:
            continue
        
        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        # Apply colormap on depth image (optional)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        
        # Stack both images horizontally
        images = np.hstack((color_image, depth_colormap))
        
        # Show images
        cv2.imshow('RealSense Camera', images)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()


In [2]:
# Install required packages
!pip install pyrealsense2 opencv-python-headless torch torchvision

import cv2
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
from torchvision.models import resnet18

# Load the pre-trained ResNet18 model for emotion detection
model = resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 7)  # 7 classes for emotions
model.eval()

# Load the RealSense camera
import pyrealsense2 as rs

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Function to preprocess image for ResNet18
def preprocess_image(image):
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.fromarray(image)
    image = preprocess(image).unsqueeze(0)
    return image

try:
    while True:
        # Wait for a color frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        
        if not color_frame:
            continue
        
        # Convert color image to numpy array
        color_image = np.asanyarray(color_frame.get_data())

        # Convert to grayscale for face detection
        gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

        # Use Haar cascade to detect faces
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Iterate through faces
        for (x, y, w, h) in faces:
            # Extract face region
            face = color_image[y:y+h, x:x+w]
            
            # Preprocess image for ResNet18
            face_tensor = preprocess_image(face)
            
            # Perform emotion detection
            with torch.no_grad():
                outputs = model(face_tensor)
                _, predicted = torch.max(outputs.data, 1)
                emotion = emotion_labels[predicted.item()]
            
            # Draw rectangle around the face
            cv2.rectangle(color_image, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
            # Put emotion label on the frame
            cv2.putText(color_image, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
        # Show the frame with emotions
        cv2.imshow('Emotion Detection', color_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\siddh\\AppData\\Roaming\\Python\\Python39\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 38.5/38.5 MB 7.7 MB/s eta 0:00:00


C:\Users\siddh\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\siddh\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\siddh/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|█████████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [00:12<00:00, 3.84MB/s]


In [10]:
# Install required packages
!pip install pyrealsense2 opencv-python-headless torch torchvision

import cv2
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
from torchvision.models import resnet18

# Load the pre-trained ResNet18 model for emotion detection
model = resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)  # 2 classes for happy and sad
model.eval()

# Load the RealSense camera
import pyrealsense2 as rs

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

# Define emotion labels
emotion_labels = ['Sad', 'Happy']

# Function to preprocess image for ResNet18
def preprocess_image(image):
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.fromarray(image)
    image = preprocess(image).unsqueeze(0)
    return image

try:
    while True:
        # Wait for a color frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        
        if not color_frame:
            continue
        
        # Convert color image to numpy array
        color_image = np.asanyarray(color_frame.get_data())

        # Convert to grayscale for face detection
        gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

        # Use Haar cascade to detect faces
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Iterate through faces
        for (x, y, w, h) in faces:
            # Extract face region
            face = color_image[y:y+h, x:x+w]
            
            # Preprocess image for ResNet18
            face_tensor = preprocess_image(face)
            
            # Perform emotion detection
            with torch.no_grad():
                outputs = model(face_tensor)
                _, predicted = torch.max(outputs.data, 1)
                emotion = emotion_labels[predicted.item()]
            
            # Draw rectangle around the face
            cv2.rectangle(color_image, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
            # Put emotion label on the frame
            cv2.putText(color_image, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
        # Show the frame with emotions
        cv2.imshow('Emotion Detection', color_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\siddh\\AppData\\Roaming\\Python\\Python39\\site-packages\\cv2\\cv2.pyd'
Check the permissions.



Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl (38.5 MB)


C:\Users\siddh\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\siddh\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Creating Dataset

In [ ]:
import os
import cv2
import numpy as np
import pyrealsense2 as rs

# Function to save image to disk
def save_image(image, label):
    # Create directory if it doesn't exist
    if not os.path.exists(label):
        os.makedirs(label)
    # Generate unique filename
    file_name = f'{label}/{label}_{len(os.listdir(label)) + 1}.jpg'
    # Save image
    cv2.imwrite(file_name, image)
    print(f"Saved image: {file_name}")

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

# Use Haar cascade to detect faces
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

try:
    while True:
        # Wait for a color frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()

        if not color_frame:
            continue

        # Convert color image to numpy array
        color_image = np.asanyarray(color_frame.get_data())

        # Convert to grayscale for face detection
        gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Display detected faces and capture images
        for (x, y, w, h) in faces:
            # Draw rectangle around the face
            cv2.rectangle(color_image, (x, y), (x+w, y+h), (255, 0, 0), 2)

            # Display image
            cv2.imshow('Capture Images - Press "h" for Happy and "s" for Sad', color_image)

            # Capture image on key press
            key = cv2.waitKey(1)
            if key == ord('h'):
                # Save image with label 'happy'
                save_image(color_image[y:y+h, x:x+w], 'happy')
            elif key == ord('s'):
                # Save image with label 'sad'
                save_image(color_image[y:y+h, x:x+w], 'sad')
            elif key == ord('q'):
                break

finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()


In [3]:
import os
import cv2
import numpy as np
import pyrealsense2 as rs

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

# Counter for filename
file_count = 1

try:
    while True:
        # Wait for a color frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()

        if not color_frame:
            continue

        # Convert color image to numpy array
        color_image = np.asanyarray(color_frame.get_data())

        # Display image
        cv2.imshow('Capture Images - Press "q" to Quit', color_image)

        # Capture image on key press
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        else:
            # Save image with sequential filename
            file_name = f'file{file_count}.jpg'
            cv2.imwrite(file_name, color_image)
            print(f"Saved image: {file_name}")
            file_count += 1

finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()


Saved image: file1.jpg
Saved image: file2.jpg
Saved image: file3.jpg
Saved image: file4.jpg
Saved image: file5.jpg
Saved image: file6.jpg
Saved image: file7.jpg
Saved image: file8.jpg
Saved image: file9.jpg
Saved image: file10.jpg
Saved image: file11.jpg
Saved image: file12.jpg
Saved image: file13.jpg
Saved image: file14.jpg
Saved image: file15.jpg
Saved image: file16.jpg
Saved image: file17.jpg
Saved image: file18.jpg
Saved image: file19.jpg
Saved image: file20.jpg
Saved image: file21.jpg
Saved image: file22.jpg
Saved image: file23.jpg
Saved image: file24.jpg
Saved image: file25.jpg
Saved image: file26.jpg
Saved image: file27.jpg
Saved image: file28.jpg
Saved image: file29.jpg
Saved image: file30.jpg
Saved image: file31.jpg
Saved image: file32.jpg
Saved image: file33.jpg
Saved image: file34.jpg
Saved image: file35.jpg
Saved image: file36.jpg
Saved image: file37.jpg
Saved image: file38.jpg
Saved image: file39.jpg
Saved image: file40.jpg
Saved image: file41.jpg
Saved image: file42.jpg
S

In [4]:
import os
import cv2
import numpy as np
import pyrealsense2 as rs

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

# Counter for filename
file_count = 1

try:
    # Prompt user for folder name
    folder_name = input("Enter folder name to save images: ")

    # Create folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created.")

    while True:
        # Wait for a color frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()

        if not color_frame:
            continue

        # Convert color image to numpy array
        color_image = np.asanyarray(color_frame.get_data())

        # Display image
        cv2.imshow('Capture Images - Press "q" to Quit', color_image)

        # Capture image on key press
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        else:
            # Save image with sequential filename in the specified folder
            file_name = os.path.join(folder_name, f'file{file_count}.jpg')
            cv2.imwrite(file_name, color_image)
            print(f"Saved image: {file_name}")
            file_count += 1

finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()


Enter folder name to save images: sad
Folder 'sad' created.
Saved image: sad\file1.jpg
Saved image: sad\file2.jpg
Saved image: sad\file3.jpg
Saved image: sad\file4.jpg
Saved image: sad\file5.jpg
Saved image: sad\file6.jpg
Saved image: sad\file7.jpg
Saved image: sad\file8.jpg
Saved image: sad\file9.jpg
Saved image: sad\file10.jpg
Saved image: sad\file11.jpg
Saved image: sad\file12.jpg
Saved image: sad\file13.jpg
Saved image: sad\file14.jpg
Saved image: sad\file15.jpg


In [9]:
import os
import cv2
import numpy as np
import pyrealsense2 as rs

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

# Counter for filename and image count
file_count = 1
image_count = 0
max_images = 500

try:
    # Prompt user for folder name
    folder_name = input("Enter folder name to save images: ")

    # Create folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created.")

    while image_count < max_images:
        # Wait for a color frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()

        if not color_frame:
            continue

        # Convert color image to numpy array
        color_image = np.asanyarray(color_frame.get_data())

        # Display image
        cv2.imshow('Capture Images - Press "q" to Quit', color_image)

        # Capture image on key press
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        else:
            # Save image with sequential filename in the specified folder
            file_name = os.path.join(folder_name, f'file{file_count}.jpg')
            cv2.imwrite(file_name, color_image)
            print(f"Saved image: {file_name}")
            file_count += 1
            image_count += 1

finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()


Enter folder name to save images: excited
Folder 'excited' created.
Saved image: excited\file1.jpg
Saved image: excited\file2.jpg
Saved image: excited\file3.jpg
Saved image: excited\file4.jpg
Saved image: excited\file5.jpg
Saved image: excited\file6.jpg
Saved image: excited\file7.jpg
Saved image: excited\file8.jpg
Saved image: excited\file9.jpg
Saved image: excited\file10.jpg
Saved image: excited\file11.jpg
Saved image: excited\file12.jpg
Saved image: excited\file13.jpg
Saved image: excited\file14.jpg
Saved image: excited\file15.jpg
Saved image: excited\file16.jpg
Saved image: excited\file17.jpg
Saved image: excited\file18.jpg
Saved image: excited\file19.jpg
Saved image: excited\file20.jpg
Saved image: excited\file21.jpg
Saved image: excited\file22.jpg
Saved image: excited\file23.jpg
Saved image: excited\file24.jpg
Saved image: excited\file25.jpg
Saved image: excited\file26.jpg
Saved image: excited\file27.jpg
Saved image: excited\file28.jpg
Saved image: excited\file29.jpg
Saved image: 

Saved image: excited\file257.jpg
Saved image: excited\file258.jpg
Saved image: excited\file259.jpg
Saved image: excited\file260.jpg
Saved image: excited\file261.jpg
Saved image: excited\file262.jpg
Saved image: excited\file263.jpg
Saved image: excited\file264.jpg
Saved image: excited\file265.jpg
Saved image: excited\file266.jpg
Saved image: excited\file267.jpg
Saved image: excited\file268.jpg
Saved image: excited\file269.jpg
Saved image: excited\file270.jpg
Saved image: excited\file271.jpg
Saved image: excited\file272.jpg
Saved image: excited\file273.jpg
Saved image: excited\file274.jpg
Saved image: excited\file275.jpg
Saved image: excited\file276.jpg
Saved image: excited\file277.jpg
Saved image: excited\file278.jpg
Saved image: excited\file279.jpg
Saved image: excited\file280.jpg
Saved image: excited\file281.jpg
Saved image: excited\file282.jpg
Saved image: excited\file283.jpg
Saved image: excited\file284.jpg
Saved image: excited\file285.jpg
Saved image: excited\file286.jpg
Saved imag

# Training on my dataset

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torchvision.models import resnet18

# Define constants
NUM_CLASSES = 4
BATCH_SIZE = 64
NUM_EPOCHS = 2

# Data augmentation and normalization for training
# Add more augmentation techniques if needed
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
train_dataset = ImageFolder(root=r'C:\Users\siddh\OneDrive\Desktop\Dataset', transform=train_transforms)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Load the pre-trained ResNet18 model
model = resnet18(pretrained=True)

# Modify the last layer to output predictions for NUM_CLASSES
model.fc = nn.Linear(512, NUM_CLASSES)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'emotion_classification_model.pth')


Epoch 1/2, Loss: 0.0969
Epoch 2/2, Loss: 0.0003


# Inference

In [34]:
import cv2
import numpy as np
import torch
from torchvision.transforms import transforms
from PIL import Image
from torchvision.models import resnet18

# Load the pre-trained ResNet18 model
model = resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 4)  # Assuming you trained the model for 4 classes (happy, sad, neutral, excited)
model.load_state_dict(torch.load('emotion_classification_model.pth'))
model.eval()

# Initialize the RealSense pipeline
import pyrealsense2 as rs
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

# Define emotion labels
emotion_labels = ['Sad', 'Happy', 'Neutral', 'Excited']

# Function to preprocess image for ResNet18
def preprocess_image(image):
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.fromarray(image)
    image = preprocess(image).unsqueeze(0)
    return image

try:
    while True:
        # Wait for a color frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        
        if not color_frame:
            continue
        
        # Convert color image to numpy array
        color_image = np.asanyarray(color_frame.get_data())

        # Convert to grayscale for face detection
        gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

        # Use Haar cascade to detect faces
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Iterate through faces
        for (x, y, w, h) in faces:
            # Extract face region
            face = color_image[y:y+h, x:x+w]
            
            # Preprocess image for ResNet18
            face_tensor = preprocess_image(face)
            
            # Perform emotion detection
            with torch.no_grad():
                outputs = model(face_tensor)
                _, predicted = torch.max(outputs.data, 1)
                emotion = emotion_labels[predicted.item()]
            
            # Draw rectangle around the face
            cv2.rectangle(color_image, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
            # Put emotion label on the frame
            cv2.putText(color_image, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
        # Show the frame with emotionsq
        cv2.imshow('Emotion Detection', color_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()
